In [1]:
# 패키지 import

from konlpy.tag import Mecab, Kkma
from collections import Counter
import math
import numpy as np

In [9]:
# 전처리

def sp(text):
    sentences = text.split(".")
    sp_words = ["'", '"', ",", ".", "?", "!", "/", "\\", "=", "↓", "↑"]
    for idx, sentence in enumerate(sentences):
        sp_idx = 0
        for sp_idx in range(len(sp_words)):
            sentences[idx-1] = sentences[idx-1].strip()
            if sp_words[sp_idx] in sentence:
                sentences[idx] = sentences[idx].replace(sp_words[sp_idx], "")
            if "" in sentences:
                sentences.remove("")
    return sentences

In [10]:
# Text Tokenizer
# 텍스트 토큰화(텍스트 전처리단계)

def token(text):
    mecab = Mecab(dicpath="C:\MeCab\mecab-ko-dic")
    nouns = mecab.nouns(text)
    return nouns
#     print(nouns)

In [11]:
# 단어 중복 배제 추출

# deduplicate_nouns = dedup_n

def dedup(nouns):
    dedup_n = []

    for noun in nouns:
    #     print(noun)
        if noun not in dedup_n:
            dedup_n.append(noun)

    # 중복 배제된 단어 추출 완료
#     print(dedup_n)
    return dedup_n

In [12]:

def tf(t, d):
    freq = np.zeros(len(t)).reshape(len(t), 1)
#     print(len(d), len(t))
    for n in range(len(t)):
        count = 0
        for m in range(len(d)):
            if t[n] in d[m]:
                count += 1
                freq[n] = count
                
    return freq

# tf test
# t = tf(dedup_n, nouns)
# print(t)

In [13]:
def idf(t, D):
    freq = np.zeros(len(t) * len(D)).reshape(len(t), len(D))
    for idx_D in range(len(D)):
        count = 0
        for idx_t in range(len(t)):
            if t[idx_t] in D[idx_D]:
                count += 1
            freq[idx_t][idx_D] = math.log(len(D)/(count+1))
    return freq

# idf test
# i = idf(dedup_n, sentences)
# print(i)

In [14]:
# tf-idf 구현

def tfidf(t, d, D):
    tfidf = tf(t, d) * idf(t, D)
    return tfidf

# tf-idf test
# ti = tfidf(dedup_n, nouns, sentences)
# print(ti)

In [15]:
# 어떻게 요약을 추출?
# 진짜 텍스트 랭크 알고리즘은 아니고, tf-idf 값이 높은 문장을  중복을 배제하여 n개만큼 추출해보았음
# 
    

def textRank():
    tr = np.zeros(len(sentences))
    idx = []
    for m in ti:
#         print(tr)
#         print(idx)
        for n in range(len(m)):
            if n == len(m)-1:
                if tr[n] < m[n]:
                    tr[n] = m[n]
            else:
                if tr[n] < m[n]:
                    tr[n+1] = tr[n]
                    tr[n] = m[n]
#     print(tr)
    li_tr = list(tr)
    temp = list(np.sort(tr))
    temp_desc = temp[::-1]
    for a in range(len(temp_desc)):
        cnt = 0
        for b in range(len(li_tr)):
            if temp_desc[a] == li_tr[b]:
                if cnt in idx:
                    b += 1
                else:
                    idx.append(cnt)
            cnt += 1
#     print(idx)
    return idx


# textrank = textRank()
# print(textrank)

In [16]:
# 요약할 텍스트 입력
while True:
    title = ""
    text = ""
    while True:
        try:
            title = input("요약할 텍스트의 타이틀을 입력해주세요\n>>> ")
            print()
            text = input("요약할 텍스트를 입력해주세요\n>>> ")
            if text and title:
                print()
                print("텍스트가 정상적으로 입력되었습니다.")
                print()
                break
            else:
                print()
                print("텍스트가 정상적으로 입력되지 않았습니다. 다시 한 번 입력해주세요")
                print()
        except:
            print("오류가 발생하였습니다")
            
            
    sentences = sp(text)
    nouns = token(text)
    dedup_n = dedup(nouns)
    ti = tfidf(dedup_n, nouns, sentences)
    textrank = textRank()

    
    try:
        n_sen = int(input(f"본문의 문장은 총 {len(sentences)}개 입니다.\n\n몇 문장으로 요약하시겠습니까?\n>>> "))
        print()
        if n_sen < len(sentences) and n_sen > 0:
            print("정상적으로 입력되었습니다")
            print()
        else:
            print(f"본문의 문장은 총 {len(sentences)}개 입니다.")
            print(f"{len(sentences)} ~ 0 사이의 숫자를 입력해주세요")
            print()
    except:
        print("오류")


    print("요약이 완료되었습니다.")
    print("본문의 요약은 다음과 같습니다.")
    print()
    print()
    print(f"제목 : {title}")
    print("-"*80)
    print("내용 :")
    sent_temp = []
    for i in range(n_sen):
        sent_temp.append(textrank[i])

    # print(sent_temp)
    sent_asc = np.sort(np.array(sent_temp))
    # print(sent_asc)

    out_temp = []
    for i in sent_asc:
        out_temp.append(sentences[i])

    out = ".".join(out_temp)
    print(out)
    print("-"*80)
    
    finish = input("종료하시려면 아무 키나 입력하세요. 엔터를 치면 종료합니다.\n")
    print()
    print()
    print()
    if not finish:
        break

요약할 텍스트의 타이틀을 입력해주세요
>>> 긁어온 글

요약할 텍스트를 입력해주세요
>>> 미국에서 코로나 19 규제가 완화되는 가운데 본격적인 여름 휴가철을 알리는 메모리얼 데이 연휴를 맞아 3천700만 명 이상이 장거리 여행에 나설 것으로 추산됐습니다.  USA투데이 등 미국 주요 언론은 현지시간 28일 전미자동차협회 발표를 인용해 이번 주말 시작되는 메모리얼 데이 연휴에 미국인 3천700만 명 이상이 집에서부터 최소 50마일, 80km 이상 떨어진 곳으로 여행할 예정이라고 전했습니다.  협회는 "코로나19 팬데믹이 한창이던 작년 메모리얼 데이 연휴와 비교해 여행객이 60% 이상 증가했다"며 팬데믹 이전인 2019년보다는 13% 적다고 설명했습니다.  여행객 대다수인 3천400만 명이 자동차를 이용한 도로 여행을 떠나고, 250만 명가량이 항공 여행을, 나머지 25만 명 정도가 버스와 기차 등 대중교통 수단을 이용할 계획입니다.  AP통신에 따르면 연휴 시작 전인 지난 27일, 미 전역의 공항에서 180만 명 이상이 항공편을 이용해 이동을 시작했습니다.  항공당국은 이번 연휴 항공편 이용객이 200만 명은 쉽게 넘을 것으로 전망하면서 "코로나19 사태가 발발한 작년 3월 이후 최고치"라고 밝혔습니다.

텍스트가 정상적으로 입력되었습니다.

본문의 문장은 총 6개 입니다.

몇 문장으로 요약하시겠습니까?
>>> 3

정상적으로 입력되었습니다

요약이 완료되었습니다.
본문의 요약은 다음과 같습니다.


제목 : 긁어온 글
--------------------------------------------------------------------------------
내용 :
여행객 대다수인 3천400만 명이 자동차를 이용한 도로 여행을 떠나고 250만 명가량이 항공 여행을 나머지 25만 명 정도가 버스와 기차 등 대중교통 수단을 이용할 계획입니다.AP통신에 따르면 연휴 시작 전인 지난 27일 미 전역의 공항에서 180만 명 이상이 항공편을 이용해 이동을 시작했